## Deliverable 2. Create a Customer Travel Destinations Map.

In [48]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [49]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Nanortalik,GL,60.1432,-45.2371,20.44,75,0,3.09
1,1,Saint-Philippe,RE,-21.3585,55.7679,78.80,65,0,4.61
2,2,Qaanaaq,GL,77.4840,-69.3632,-8.28,75,100,8.39
3,3,Severodvinsk,RU,64.5635,39.8302,1.40,66,75,11.18
4,4,Ostrovnoy,RU,68.0531,39.5131,12.63,91,100,12.33


In [50]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [53]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Saint-Philippe,RE,-21.3585,55.7679,78.80,65,0,4.61
6,6,Coihaique,CL,-45.5752,-72.0662,84.20,32,0,6.91
8,8,Georgetown,MY,5.4112,100.3354,78.80,78,20,3.44
10,10,Mahebourg,MU,-20.4081,57.7000,78.80,78,40,6.91
13,13,Miandrivazo,MG,-19.5167,45.4667,78.85,83,100,3.85
15,15,Kumasi,GH,6.6885,-1.6244,80.58,71,69,6.87
16,16,Khorixas,NaN,-20.3667,14.9667,74.68,50,59,8.14
17,17,Tarauaca,BR,-8.1614,-70.7656,76.46,95,96,4.65
18,18,Hambantota,LK,6.1241,81.1185,77.56,75,100,14.72
22,22,Hithadhoo,MV,-0.6000,73.0833,82.42,77,96,11.30


In [54]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID       0
City          0
Country       6
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [56]:
clean_df.count()

City_ID       204
City          204
Country       198
Lat           204
Lng           204
Max Temp      204
Humidity      204
Cloudiness    204
Wind Speed    204
dtype: int64

In [58]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Saint-Philippe,RE,-21.3585,55.7679,78.80,65,0,4.61
6,6,Coihaique,CL,-45.5752,-72.0662,84.20,32,0,6.91
8,8,Georgetown,MY,5.4112,100.3354,78.80,78,20,3.44
10,10,Mahebourg,MU,-20.4081,57.7000,78.80,78,40,6.91
13,13,Miandrivazo,MG,-19.5167,45.4667,78.85,83,100,3.85
15,15,Kumasi,GH,6.6885,-1.6244,80.58,71,69,6.87
17,17,Tarauaca,BR,-8.1614,-70.7656,76.46,95,96,4.65
18,18,Hambantota,LK,6.1241,81.1185,77.56,75,100,14.72
22,22,Hithadhoo,MV,-0.6000,73.0833,82.42,77,96,11.30
32,32,Touros,BR,-5.1989,-35.4608,80.60,69,0,9.22


In [60]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,78.80,-21.3585,55.7679,
6,Coihaique,CL,84.20,-45.5752,-72.0662,
8,Georgetown,MY,78.80,5.4112,100.3354,
10,Mahebourg,MU,78.80,-20.4081,57.7000,
13,Miandrivazo,MG,78.85,-19.5167,45.4667,
15,Kumasi,GH,80.58,6.6885,-1.6244,
17,Tarauaca,BR,76.46,-8.1614,-70.7656,
18,Hambantota,LK,77.56,6.1241,81.1185,
22,Hithadhoo,MV,82.42,-0.6000,73.0833,
32,Touros,BR,80.60,-5.1989,-35.4608,


In [62]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
     print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [68]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df =pd.DataFrame(hotel_df)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,78.80,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
6,Coihaique,CL,84.20,-45.5752,-72.0662,Diego De Almagro Coyhaique
8,Georgetown,MY,78.80,5.4112,100.3354,Cititel Penang
10,Mahebourg,MU,78.80,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
13,Miandrivazo,MG,78.85,-19.5167,45.4667,Hôtel Arc En Ciel Miandrivazo
15,Kumasi,GH,80.58,6.6885,-1.6244,Okumah Hotels | Kumasi
17,Tarauaca,BR,76.46,-8.1614,-70.7656,Flat Temporada
18,Hambantota,LK,77.56,6.1241,81.1185,Bungalow 63
22,Hithadhoo,MV,82.42,-0.6000,73.0833,Scoop Guest House
32,Touros,BR,80.60,-5.1989,-35.4608,INN NEW HORIZON


In [70]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [73]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [72]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))